In [1]:
from pyhive import hive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# create a connection to the hive server
conn = hive.Connection(host='localhost', port=10000, username='hive', database='fashion_db', auth='NOSASL')

# create a cursor
cursor = conn.cursor()

In [ ]:
# create a database
cursor.execute('CREATE DATABASE IF NOT EXISTS fashion_db')

In [ ]:
# create a table in the database
cursor.execute('CREATE TABLE IF NOT EXISTS fashion_db.fashion (brand string, product_name string, price float, rating float, review_count int, product_url string, image_url string, product_id string) ROW FORMAT DELIMITED FIELDS TERMINATED BY "," LINES TERMINATED BY "" STORED AS TEXTFILE')

# create a table in the database
cursor.execute('CREATE TABLE IF NOT EXISTS fashion_db.brand (brand_name string, brand_url string, brand_id string) ROW FORMAT DELIMITED FIELDS TERMINATED BY "," LINES TERMINATED BY "" STORED AS TEXTFILE')

# load the data into the tables
cursor.execute('LOAD DATA LOCAL INPATH "/content/fashiondataset.csv" INTO TABLE fashion_db.fashion')
cursor.execute('LOAD DATA LOCAL INPATH "/content/fashion_brand_details.xlsx" INTO TABLE fashion_db.brand')

In [ ]:
# check if the tables are created in the database
cursor.execute('SHOW TABLES')
cursor.fetchall()

In [ ]:
# select from the fashion table
cursor.execute('SELECT * FROM fashion_db.fashion LIMIT 5')
cursor.fetchall()

# select from the brand table
cursor.execute('SELECT * FROM fashion_db.brand LIMIT 5')
cursor.fetchall()

In [ ]:
# select data from both tables and create a main dataframe to be used for analysis
cursor.execute('SELECT * FROM fashion_db.fashion INNER JOIN fashion_db.brand ON fashion_db.fashion.brand = fashion_db.brand.brand_name')
main_df = pd.DataFrame(cursor.fetchall())

In [ ]:
# close the connection
conn.close()

### Data Analysis with Apache Hive

In [ ]:
# create a pandas dataframe from the main dataframe
main_df_hive = main_df.toPandas()

# display the first 5 rows of the pandas dataframe
main_df_hive.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
# What is the most popular brand?
plt.figure(figsize=(10,8))
sns.countplot(x='brand', data=main_df_hive, order=main_df_hive['brand'].value_counts().index[:10],palette='viridis',orient='h')

plt.title('The most popular brand',fontsize=20)
plt.xlabel('Brand',fontsize=15)
plt.ylabel('frequency',fontsize=15)
plt.xticks(rotation=90,fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
sorted_df = main_df_hive.sort_values(by='price', ascending=False)

In [ ]:
# what are the most expensive brands?
plt.figure(figsize=(10,8))
sns.barplot(x=main_df_hive['brand'], y=sorted_df['price'] , data=main_df_hive ,order=main_df_hive['brand'].value_counts().index[:10],palette='viridis')
plt.title("The Most Expensive brands",fontsize=20)
plt.xlabel("Brands",fontsize=15)
plt.ylabel("Prices",fontsize=15)
plt.xticks(rotation=85,fontsize=12)
plt.yticks(fontsize=12)
plt.show()